In [ ]:
from google.colab import drive
import os
import sys
import shutil
import subprocess

drive.mount('/content/drive')

PROJECT_DIR = '/content/drive/MyDrive/SVGEditor'
CODE_PATH = f"{PROJECT_DIR}/code"
DATA_PATH = f"{PROJECT_DIR}/data"
TEST_PATH = f"{PROJECT_DIR}/test"

In [ ]:
# 1. Clean existing code directory and clone fresh repository
if os.path.exists(CODE_PATH):
    shutil.rmtree(CODE_PATH)

os.chdir(PROJECT_DIR)
result = subprocess.run(['git', 'clone', 'https://github.com/huanbasara/SVGEditor.git', 'code'],
                       capture_output=True, text=True)
print(f"Code repository {'successfully' if result.returncode == 0 else 'failed'} to {CODE_PATH}{f', {result.stderr}' if result.returncode != 0 else ''}")

# 2. Display latest commit information
os.chdir(CODE_PATH)
commit_info = subprocess.run(['git', 'log', '-1', '--pretty=format:%H|%ci|%s'],
                           capture_output=True, text=True)

if commit_info.returncode == 0:
    hash_code, commit_time, commit_msg = commit_info.stdout.strip().split('|', 2)
    print(f"Latest commit:\n   Hash: {hash_code[:8]}\n   Time: {commit_time}\n   Message: {commit_msg}")

# 3. Add code path to Python sys.path so we can import our modules
if CODE_PATH not in sys.path:
    sys.path.insert(0, CODE_PATH)
    print(f"✅ Added {CODE_PATH} to Python path")

# 4. Clear custom modules from cache
modules_to_clear = [

]

for base in modules_to_clear:
    to_remove = [m for m in sys.modules if m.startswith(base)]
    for m in to_remove:
        del sys.modules[m]

print("✅ Modules reloaded!")

In [ ]:
%pip install diffusers accelerate safetensors transformers huggingface_hub

In [ ]:
# ========================================
# 模型下载Cell - 只运行一次，下载后可以注释掉
# ========================================

from huggingface_hub import snapshot_download
import os

MODEL_PATH = f"{PROJECT_DIR}/models"

def download_models():
    """下载并缓存instruct-pix2pix模型到Google Drive"""
    models_config = {
        "instruct-pix2pix": "timbrooks/instruct-pix2pix"
    }

    print("🚀 Starting model download process...")
    print(f"📁 Model cache directory: {MODEL_PATH}")

    # 创建模型目录
    os.makedirs(MODEL_PATH, exist_ok=True)

    for model_name, model_id in models_config.items():
        model_dir = f"{MODEL_PATH}/{model_name}"

        if not os.path.exists(model_dir):
            print(f" Downloading {model_name}...")
            try:
                snapshot_download(
                    repo_id=model_id,
                    local_dir=model_dir
                )
                print(f"✅ {model_name} downloaded successfully")
            except Exception as e:
                print(f"❌ Failed to download {model_name}: {e}")
        else:
            print(f"✅ {model_name} already cached")

    print(" Model ready!")

# 执行下载
download_models()